# Clustering at state level

## imports

In [13]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import PowerTransformer
from utils import add_Loss, clean_data_state

from pyclustering.utils.metric import distance_metric, type_metric
from pyclustering.cluster.kmeans import kmeans, kmeans_visualizer
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer

## Import and clean data

In [46]:
#Select the dataset of one season of one year
YEAR = 2019
SEASON = "Rabi" # or "Kharif" 

#Path to the dataset
pathData = f"Data\RawDataUnified\RawData_{YEAR}_{SEASON}"
print(pathData)


df = pd.read_csv(pathData)

Data\RawDataUnified\RawData_2019_Rabi


C:\Users\marti\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [47]:
#Clean an add loss to data
df=add_Loss(clean_data_state(df))
df

,State,Crop,Area Sown (Ha),Area Insured (Ha),SI Per Ha (Inr/Ha),Sum Insured (Inr),Indemnity Level,2002 Yield,2003 Yield,2004 Yield,...,2009 Yield,2010 Yield,2011 Yield,2012 Yield,2013 Yield,2014 Yield,2015 Yield,2016 Yield,2017 Yield,Loss
key,,,,,,,,,,,,,,,,,,,,,
andhra pradesh_anantapur_vidapanakal___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,1154.000000,760.00000,561.000000,816.000000,766.000000,670.000000,67.000000,202.000000,746.000000,1.078638e+05
andhra pradesh_anantapur_vajrakarur___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,1154.000000,944.00000,703.000000,842.000000,775.000000,647.000000,450.000000,627.000000,375.000000,0.000000e+00
andhra pradesh_anantapur_gooty___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,1154.000000,752.00000,504.000000,1320.000000,1223.000000,688.000000,814.000000,379.000000,661.000000,1.516659e+06
andhra pradesh_anantapur_guntakal___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,1154.000000,752.00000,504.000000,1320.000000,1223.000000,688.000000,814.000000,379.000000,661.000000,1.516659e+06
andhra pradesh_anantapur_pamidi___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,1154.000000,752.00000,504.000000,1320.000000,1223.000000,688.000000,814.000000,379.000000,661.000000,1.516659e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
west bengal_howrah_uluberia-ii___,12,Wheat,204.149503,0.200000,56810.0,1.136200e+04,0.9,2275.972276,4629.899829,4377.946588,...,2084.770000,1987.31000,1417.770000,2804.980000,2382.030000,2518.480000,1362.950000,2469.040109,2766.065757,0.000000e+00
west bengal_purba medinipur_haldia___,12,Wheat,204.149503,0.750000,38557.0,2.891775e+04,0.9,2275.972276,4629.899829,4377.946588,...,2450.120000,2569.92000,2431.370000,2564.580000,2623.860000,2756.520000,2496.650000,2469.040109,2766.065757,0.000000e+00
west bengal_purba medinipur_mohisadal-ii___,12,Wheat,204.149503,0.750000,38557.0,2.891775e+04,0.9,2275.972276,4629.899829,4377.946588,...,2450.120000,2569.92000,2431.370000,2564.580000,2623.860000,2756.520000,2496.650000,2469.040109,2766.065757,0.000000e+00


In [48]:
# Définition d'une distance entre deux parcelles avec forte pénalisation si état différent

def distance_state_penalized(point1, point2):
    dimension = len(point1)
    if point1[0] == point2[0]:
        result = 0.0
    else:
        result = 99999999999999
    p1_numpy = np.array(point1[1:])
    p2_numpy = np.array(point2[1:])
    result += np.linalg.norm(p1_numpy-p2_numpy)
    return result

metric_perso = distance_metric(type_metric.USER_DEFINED, func=distance_state_penalized)

## K-Means

### Preprocessing

In [49]:
data = pd.get_dummies( df, columns=["Crop"])

for c in data.columns:
    pt = PowerTransformer()
    data.loc[:, c] = pt.fit_transform(np.array(data[c]).reshape(-1, 1))

data_list =data.to_numpy()

### Algorithme

In [50]:
# Nombre de clusters
k = 8

# Prepare initial centers using K-Means++ method.
initial_centers = kmeans_plusplus_initializer(data_list, k).initialize()
 
# Create instance of K-Means algorithm with prepared centers.
kmeans_instance = kmeans(data_list, initial_centers,metric = metric_perso)
 
# Run cluster analysis and obtain results.
kmeans_instance.process()
clusters = kmeans_instance.get_clusters()
final_centers = kmeans_instance.get_centers()

In [51]:
tab_clusters = np.zeros(len(df)) #tab_clusters[i] renvoit le numéro de cluster de la i-ème ligne
for i in range(k):
    for x in clusters[i]:
        tab_clusters[x] = i
tab_clusters

array([0., 0., 0., ..., 2., 2., 2.])

In [52]:
df2 = df.copy(deep = True)
df2["Cluster"] = tab_clusters
df2

,State,Crop,Area Sown (Ha),Area Insured (Ha),SI Per Ha (Inr/Ha),Sum Insured (Inr),Indemnity Level,2002 Yield,2003 Yield,2004 Yield,...,2010 Yield,2011 Yield,2012 Yield,2013 Yield,2014 Yield,2015 Yield,2016 Yield,2017 Yield,Loss,Cluster
key,,,,,,,,,,,,,,,,,,,,,
andhra pradesh_anantapur_vidapanakal___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,760.00000,561.000000,816.000000,766.000000,670.000000,67.000000,202.000000,746.000000,1.078638e+05,0.0
andhra pradesh_anantapur_vajrakarur___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,944.00000,703.000000,842.000000,775.000000,647.000000,450.000000,627.000000,375.000000,0.000000e+00,0.0
andhra pradesh_anantapur_gooty___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,752.00000,504.000000,1320.000000,1223.000000,688.000000,814.000000,379.000000,661.000000,1.516659e+06,0.0
andhra pradesh_anantapur_guntakal___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,752.00000,504.000000,1320.000000,1223.000000,688.000000,814.000000,379.000000,661.000000,1.516659e+06,0.0
andhra pradesh_anantapur_pamidi___,0,Bengal Gram (Chana),204.149503,77.473065,30000.0,7.277487e+06,0.8,2275.972276,4629.899829,4377.946588,...,752.00000,504.000000,1320.000000,1223.000000,688.000000,814.000000,379.000000,661.000000,1.516659e+06,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
west bengal_howrah_uluberia-ii___,12,Wheat,204.149503,0.200000,56810.0,1.136200e+04,0.9,2275.972276,4629.899829,4377.946588,...,1987.31000,1417.770000,2804.980000,2382.030000,2518.480000,1362.950000,2469.040109,2766.065757,0.000000e+00,2.0
west bengal_purba medinipur_haldia___,12,Wheat,204.149503,0.750000,38557.0,2.891775e+04,0.9,2275.972276,4629.899829,4377.946588,...,2569.92000,2431.370000,2564.580000,2623.860000,2756.520000,2496.650000,2469.040109,2766.065757,0.000000e+00,2.0
west bengal_purba medinipur_mohisadal-ii___,12,Wheat,204.149503,0.750000,38557.0,2.891775e+04,0.9,2275.972276,4629.899829,4377.946588,...,2569.92000,2431.370000,2564.580000,2623.860000,2756.520000,2496.650000,2469.040109,2766.065757,0.000000e+00,2.0


In [54]:
for i in range(13):
    print(np.bincount(df2[df2["State"] == i]["Cluster"].to_numpy().astype(int)).argmax())

2
0
2
2
6
2
0
2
0
2
2
0
2
